# Egyirányú n-gram-modellek

## Modell betöltése

In [1]:
import pickle

import ngram_model
from ngram_model import MultiNgramTrieRoot, BiMultiNgramModel

with open('ngram_forward.model', 'rb') as savefile:
    trie_forward = pickle.load(savefile)
with open('ngram_backward.model', 'rb') as savefile:
    trie_backward = pickle.load(savefile)

## A trie-csúcsokban tárolt információk

In [2]:
print(trie_forward.total)
print(trie_backward.total)

1877940021
1877940021


Ennyi karakter volt összesen a tanítókorpuszban.

In [3]:
print(trie_forward.children.keys())

dict_keys(['\x00', 'H', 'V', 'G', ' ', '9', '/', '0', '8', '.', 's', 'z', 'á', 'm', '1', 'f', 'e', 'b', 'r', 'u', '2', '7', '\n', 'A', 'ű', 'n', 'ü', 'l', 'd', 'ö', 'é', ',', 'a', 'y', 'k', 'o', 't', 'v', 'i', 'í', 'g', 'ő', 'c', 'h', 'j', 'ó', '-', 'p', 'M', '"', ':', '(', ')', 'ú', 'O', 'J', 'B', 'P', 'D', '3', 'R', 'F', 'K', 'C', 'E', 'T', 'L', '6', 'x', '5', 'I', 'N', 'S', 'Z', 'Ő', '4', 'É', 'Í', 'Á', 'U', ';', 'Ú', 'Y', 'w', 'Ü', "'", 'X', '=', '*', 'W', '?', 'Ö', 'Ó', '!', 'q', 'ä', '„', '”', 'Q', 'Ű', '&', '[', ']', 'ë', '\x01', '+', '@', '–', '%', '>', '•', '|', '$', '_'])


A tanítás közben (lásd `MultiNgramTrieRoot.main()`) a tanítósztringre meghívtam a modell példányszintű `MultiNgramTrieRoot.replace_oov()` metódusát, ami az inputsztringben kicserélte az LSTM-modell outputkódolója szerint OVV-karakternek számító karaktereket OOV-karakterekre. Ez értelemszerűen azért pont így történt, hogy közvetlenül összevethető legyen az n-gram- és az LSTM-modell, de nyilván ettől függetlenül is lehet értelme a nagyon ritka karaktereket OOV-ként kezelni. Az `\u0000` a paddingkarakter, az `\u0001` az OOV-karakter.

Hogy pontosan mi számít ritka karakternek, az a `encode_characters.py` `main()` függvényéből derül ki. Ott az `INPUT_COUNT_THRESHOLD` és a `FREQ_EXPONENT` értékét pedig úgy határoztam meg, hogy futtattam a teljes tanítókorpuszra a `press_corpus_scripts` repóból a `list_chars_in_files.py` szkriptet a korpuszra, sokáig nézegettem ennek az outputját, aztán húztam önkényesen egy határt, hogy mit különböztetek még meg külön karakterként és mi legyen már OOV.

In [4]:
print(trie_forward.children['a'].total)
print(trie_forward.prefix_count('a'))
print(trie_forward.children['a'].children['b'].total)
print(trie_forward.prefix_count('ba'))

132188621
132188621
6497044
6497044


Ennyi 'a' unigram, illetve 'ba' bigram van a korpuszban, és így tároltam a számokat.

## Simított n-gram-valószínűségek, backoff

In [5]:
trie_forward.estimate_alternatives('macsk')

{'\x00': 2.356742658931507e-08,
 'H': 2.2296633979106903e-10,
 'V': 3.4195873874692456e-10,
 'G': 5.094723100925464e-10,
 ' ': 0.003946694415040005,
 '9': 1.213029309744158e-10,
 '/': 5.531413652433361e-09,
 '0': 1.0512920684449368e-10,
 '8': 8.086862064961053e-11,
 '.': 0.0003115811380294741,
 's': 0.00010386037934315803,
 'z': 2.9130558070033216e-05,
 'á': 0.0005193018967157901,
 'm': 6.744061724394456e-07,
 '1': 2.0725472206485897e-09,
 'f': 0.00020772075868631605,
 'e': 0.008101109588766326,
 'b': 0.00020772075868631605,
 'r': 0.0006231622760589482,
 'u': 0.0021810679662063186,
 '2': 1.1067448368903842e-09,
 '7': 9.819761078881278e-11,
 '\n': 4.707015828211045e-08,
 'A': 1.3828534131083401e-09,
 'ű': 8.143123519612996e-08,
 'n': 0.004362135932412637,
 'ü': 6.922030453124063e-07,
 'l': 3.2506643915923016e-07,
 'd': 1.0713012757256691e-07,
 'ö': 0.00020772075868631605,
 'é': 0.0010386037934315801,
 ',': 0.0011424641727747382,
 'a': 0.009970596416943171,
 'y': 7.503799310077362e-08,
 

In [6]:
print(sum(trie_forward.estimate_alternatives('macsk').values()))

1.0000000000000007


Ez egy olyan dictionaryt add vissza, aminek a kulcsai a lehetséges utolsó betűk, az értékei pedig az utolsó betűknek a modell szerinti feltételes valószínűségei. Azokra az utolsó betűkre, amik a kezdő (n-1)-gram után 0 alkalommal fordultak elő, a 0 előfordulás-szám helyett a végső bigram simított valószínűségét használja előfordulásszámként (ami mindenképp kisebb, mint 0, de a végső bigramtól függ, hogy mennyivel), tehát mindenképp ritkábbnak becsülöm, mint a ténylegesen előforduló alternatíva-n-gramok darabszámát.

Például: a *macsk@* előfordulás-száma 0, így a *k@* bigram feltételes valószínűségét használom becsült előfordulásszámként.

Lehetne ugye itt is a bigram helyett az (n-1)-gramhoz backoffolni, de csak az a lényeg, hogy ez egy kellően kis szám legyen, ezért nagyjából teljesen mindegy, a bigram is megteszi.

In [7]:
print(trie_forward.prefix_count('macsk@'))
print(trie_forward.bigram_freqs['k@'])

0
2.8690037067041686e-06


Mindenesetre a tényleges előfordulásszámokat és az így becsült előfordulásszámokat összeadom az összes alternatívára, és normalizálom az egészet 1-re.

In [8]:
print(trie_forward.prefix_count('macsk'))
print(trie_forward.prefix_count('macsó'))
print(trie_forward.prefix_count('macse'))
print(trie_forward.prefix_count('macs-'))
# ...

7620
757
78
6


Backoff az n-1-gramra akkor van, ha az **összes** alternatíva előfordulásszáma 0. Például becsüljük a *macút* 5-gram valószínűségét. *macú* nincs, ezért backoffol a 4-gram-modellre. *acú* adott esetben van.

In [9]:
print(trie_forward.prefix_count('macú'))
print(trie_forward.prefix_count('acú'))

0
42


In [10]:
trie_forward.estimate_alternatives('acút')

{'\x00': 2.4285701398526387e-06,
 'H': 2.11180012161099e-08,
 'V': 1.5838500912082427e-08,
 'G': 1.055900060805495e-08,
 ' ': 0.7772882601512991,
 '9': 3.875462538059061e-12,
 '/': 2.9037251672151113e-07,
 '0': 5.279500304027475e-09,
 '8': 2.7183997592530944e-12,
 '.': 4.0187556314257146e-05,
 's': 0.023554189701554522,
 'z': 0.0008910635023131492,
 'á': 3.795960718595755e-06,
 'm': 1.3916762801416425e-05,
 '1': 5.279500304027475e-09,
 'f': 5.50704676713106e-05,
 'e': 3.880432723460194e-06,
 'b': 0.00017101885334836202,
 'r': 0.0009585196776977083,
 'u': 2.7981351611345616e-07,
 '2': 3.1677001824164854e-08,
 '7': 5.279500304027475e-09,
 '\n': 4.846581279097222e-06,
 'A': 8.44720048644396e-08,
 'ű': 6.335400364832971e-08,
 'n': 0.023554189701554522,
 'ü': 5.068320291866377e-07,
 'l': 0.004416476227829016,
 'd': 0.00020959616206989076,
 'ö': 1.8478251064096165e-07,
 'é': 5.4695623149724645e-06,
 ',': 0.09421675880621809,
 'a': 0.00015228190676936848,
 'y': 5.279500304027475e-08,
 'k': 0.

Az egész filozófiája akkor látszik jobban, ha olyan n-gramot veszünk, ahol a ténylegesen előforduló alternatívák száma nagyon kicsi:

In [11]:
print(trie_forward.prefix_count('macz'))
print(trie_forward.prefix_count('maczo'))

1
1


Az béna nyelvmodell lenne, ami ilyenkor azt mondja, hogy a *maczo* valószínűsége 1, az összes többié 0 vagy valami elenyésző kicsi szám. A használt simítási eljárással (a *za*, *zb* stb. bigramok valószínűségével becsüljük 0 előfordulású 5-gramok előfordulás-számát) normalizálás után sokkal értelmesebb számokat kapunk:

In [12]:
trie_forward.estimate_alternatives('maczó')

{'\x00': 5.229501968233869e-05,
 'H': 1.4106556369031632e-06,
 'V': 7.983380802254165e-07,
 'G': 8.060889353732362e-07,
 ' ': 0.09028946358958553,
 '9': 6.200684118255663e-08,
 '/': 4.40248572396152e-06,
 '0': 2.4027650958240696e-07,
 '8': 3.100342059127832e-07,
 '.': 0.0018609570684260375,
 's': 0.00926641860914848,
 'z': 0.007809412858461355,
 'á': 0.041357098157142336,
 'm': 0.0030530695935812794,
 '1': 9.223517625905299e-07,
 'f': 0.0005735090249526141,
 'e': 0.09277016353392092,
 'b': 0.0033183658635808485,
 'r': 0.0018402080291953244,
 'u': 0.003134639593156933,
 '2': 2.7825569980672285e-06,
 '7': 1.47266247808572e-07,
 '\n': 0.00010470630219189469,
 'A': 2.146986875946023e-06,
 'ű': 0.0007967104006443744,
 'n': 0.009385122955737337,
 'ü': 0.008808583346421926,
 'l': 0.006233943037694956,
 'd': 0.007611293250027939,
 'ö': 0.011505036094652027,
 'é': 0.022048392587693484,
 ',': 0.0038965951593184847,
 'a': 0.03824595915679359,
 'y': 0.00032281536605153767,
 'k': 0.0042303857362593

Csak összehasonlításképp így nézne ki (a *maczo* 1-ének hozzáadása a normalizálás előtt), ha nem azonnal a bigram-modellre, hanem a 4-gram-modellre lépnénk vissza:

In [13]:
alts = trie_forward.estimate_alternatives('aczó')
# maczo-ból 1 van
alts['o'] = 1
# normalizálás
counts_sum = sum(alts.values())
alts = {k: v / counts_sum for k, v in alts.items()}

In [14]:
alts

{'\x00': 9.150216774292263e-09,
 'H': 2.468266567246468e-10,
 'V': 1.3968761342109133e-10,
 'G': 1.4104380384265534e-10,
 ' ': 0.020015446009443764,
 '9': 9.015966670920614e-05,
 '/': 7.703161594483483e-10,
 '0': 4.20419030684838e-11,
 '8': 5.424761686255975e-11,
 '.': 0.00018031933341841228,
 's': 0.00027047900012761847,
 'z': 1.3664364401989094e-06,
 'á': 0.004147344668623483,
 'm': 0.008294689337246966,
 '1': 1.6138666016611522e-10,
 'f': 0.00018031933341841228,
 'e': 0.00703245400331808,
 'b': 0.00027047900012761847,
 'r': 3.219867298877234e-07,
 'u': 0.002434311001148566,
 '2': 4.868723613414737e-10,
 '7': 2.5767618009715876e-11,
 '\n': 1.832077640490799e-08,
 'A': 3.7566474677322617e-10,
 'ű': 1.394028134325629e-07,
 'n': 0.0005409580002552369,
 'ü': 1.5412643036331454e-06,
 'l': 1.090771750350126e-06,
 'd': 1.3317708568333121e-06,
 'ö': 2.0130707455814647e-06,
 'é': 0.00351622700165904,
 ',': 0.002253991667730153,
 'a': 0.012712513005998067,
 'y': 0.15615654274034504,
 'k': 0.14

Úgy voltam vele, hogy miután ezt a rekurzív struktúrában sokkal költségesebb lenne mindig kiszámolni, másrészt annyira drasztikusan nem tér el a bigramos közelítéstől (bár az utóbbi adott esetben persze csúnyán alábecsüli a *czy* és a *czk* valószínűségét), nem éri meg belefektetni a plusz számolási munkát és az algoritmus bonyolítását.

Mindenesetre visszatérve a backoffra: ha a kezdő 3-gram sem fordult elő, visszalépünk a trigram-modellre, ha pedig az sem létezne, akkor a bigram-modellre. Becsüljük a *zxűxt* 5-gram valószínűségét:

In [15]:
print(trie_forward.prefix_count('zxűx'))
print(trie_forward.prefix_count('xűx'))
print(trie_forward.prefix_count('űx'))
print(trie_forward.bigram_freqs['xt'])

0
0
0
0.05202814798238691


A teljes bigram-modell (tehát az ábécében szereplő összes karakter kombinációja) már eleve simítva készen el van tárolva, hogy ennyivel gyorsabban fusson a modell. (Az elő nem forduló bigramoknál simán összeszoroztam a két unigram valószínűségét.)

In [16]:
print(len(trie_forward.children.keys()))
print(len(trie_forward.children.keys()) ** 2)
print(len(trie_forward.bigram_freqs.keys()))

114
12996
12996


A 114 az az LSTM-beli 113 outputkarakter plusz a padding. Az LSTM soha nem jelez előre paddinget a tanítás menete miatt, hanem az csak kontextusként funkcionál, az n-gram-modellben viszont értelemszerűen megegyezik az output (utolsó karakter) és az input (a feltétel) ábécéje, így ezzel az eggyel több karakter van az ábécéjében. Kis szépséghiba, hogy a betanításkor paddingre végződő n-gramokra is tanítottam a modellt (nem csak a vele kezdődőkre), tehát ebben eltér az n-gram-modell tanítása az LSTM-modellétől, de nagyjából teljesen mindegy.

## Kényelmi metódusok

Az `estimate_prob()` az n-gram utolsó karakterének becsült (lényegében maximum likelihood-becslés alapján + a 0 előfordulású alternatívák simítása) feltételes valószínűségét adja vissza adott n-1 hosszú prefixum után.

In [17]:
trie_forward.estimate_prob('macsk')

0.7914160905948642

Ugyanaz, mint:

In [18]:
trie_forward.estimate_alternatives('macsk')['k']

0.7914160905948642

Az `estimate_alternatives()` az ábécé szerint lehetséges összes utolsó karakter feltételes valószínűségét adja vissza dictionary formájában.

Ha túl hosszú n-gramot adunk meg, akkor az n hosszúságú szuffixuma alapján becsül.

In [19]:
print(trie_forward.estimate_prob('macskákat', verbose=True))
print(trie_forward.estimate_prob('kákat'))

macskákat is longer than the model's order, estimating for acskákat
macskáka not seen in corpus, estimating based on acskáka
acskákat is longer than the model's order, estimating for cskákat
acskáka not seen in corpus, estimating based on cskáka
cskákat is longer than the model's order, estimating for skákat
cskáka not seen in corpus, estimating based on skáka
skákat is longer than the model's order, estimating for kákat
0.9978041429885643
0.9978041429885643


In [20]:
trie_forward.estimate_alternatives('macskákat')['t']

0.9978041429885643

Ha 0 gyakoriságú a feltétel (tehát az összes alternatív 5-gram gyakorisága 0 a tanítókorpuszban), akkor backoffol, amíg nem jut el egy nem 0 gyakoriságúig vagy a bigramig:

In [21]:
print(trie_forward.estimate_prob('zxűxt', verbose=True))
print(trie_forward.estimate_prob('xt', verbose=True))
print(trie_forward.estimate_alternatives('zxűxt')['t'])

zxűx not seen in corpus, estimating based on xűx
xűx not seen in corpus, estimating based on űx
űx not seen in corpus, estimating based on x
0.05202814798238691
0.05202814798238691
0.05202814798238689


Ha n-nél rövidebb sztringet adunk meg, akkor szintén backoffol impliciten a megfelelő rendű n-gram-modellre:

In [22]:
print(trie_forward.estimate_prob('cska', verbose=True))
print(trie_forward.estimate_alternatives('cska')['a'])

0.1979940120293708
0.1979940120293708


In [23]:
print(trie_forward.estimate_prob('ska', verbose=True))
print(trie_forward.estimate_alternatives('ska')['a'])

0.07334865741519339
0.07334865741519339


A `predict_next()` a megadott sztring **után következő** lehetséges karakterek közül csökkenő sorrendben az m legvalószínűbbet adja vissza listaként.

In [24]:
print(trie_forward.predict_next('macs'))
print(trie_forward.predict_next('macs', 4))

['k']
['k', 'ó', 'ú', 'o']


A `metrics on string()` végigléptet egy n hosszúságú ablakot az inputsztringen, és kiszámítja a sztringre a karakterenkénti perplexitást és a `predict_next()` által jósolt legvalószínűbb karakter pontosságát (tehát ténylegesen az-e a következő karakter vagy nem).

Opcionális egy-egy paddingkaraktert is ad hozzá az elejéhez és a végéhez, ebben az esetben az első karakter valószínűségét a padding után a bigram-modell alapján számolja, aztán a 2. karakter valószínűségét a padding és az 1. után a trigram-modell alapján stb., majd a sztring végéig az összes teljes n-gramét, ami adott esetben 5-gram.

Ha nincs padding, akkor csak a teljes 5-gramokat számolja, tehát az első 4 karakterre nem számít becsült valószínűséget és jósolt karaktert.

In [25]:
long_string_1 = '''A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.'''
long_string_2 = '''Tavaly októberben adtunk hírt róla, hogy Mészáros Lőrinc korábbi felesége, Kelemen Beatrix az egykori televíziós műsorvezetővel, Joshi Bharattal alkot egy párt. Most viszont a Story alig pár hónappal később arról számolt be, hogy véget ért kettejük kapcsolata, a lap információi szerint Joshi Bharat már el is költözött abból a budai villából, ahol eddig együtt éltek.
Kelemen Beatrix hosszú évekig élt házasságban Mészáros Lőrinccel. A pár 2020 őszén adta be a váló pert, a részletekről, így a vagyonról is hamar megegyeztek. Egy évvel később már arról lehetett hallani, hogy Kelemen Beatrix Joshi Bharattal alkot egy párt.
Közben pedig, ki ne tudná, Mészáros Lőrinc is újra szerelmes lett és már újra is nősült. Az ország egyik leggazdagabb embere a TV2 korábbi műsorvezetőjével, Várkonyi Andreával kötötte össze az életét.'''
print(long_string_1)
print()
print(long_string_2)

A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.

Tavaly októberben adtunk hírt róla, hogy Mészáros Lőrinc korábbi felesége, Kelemen Beatrix az egykori televíziós műsorvezetővel, Joshi Bhara

In [26]:
%%time
print(len(long_string_1))
print(trie_forward.metrics_on_string(long_string_1))
print(trie_forward.metrics_on_string(long_string_1, padded=True))

858
(0.5831381733021077, 3.744253215919672)
(0.5820721769499418, 3.7884149831881415)
CPU times: user 680 ms, sys: 215 µs, total: 681 ms
Wall time: 677 ms


In [27]:
%%time
print(len(long_string_2))
print(trie_forward.metrics_on_string(long_string_2))
print(trie_forward.metrics_on_string(long_string_2, padded=True))

825
(0.5152253349573691, 4.328136190221757)
(0.5145278450363197, 4.335518273647436)
CPU times: user 658 ms, sys: 3.8 ms, total: 662 ms
Wall time: 660 ms


Lehet a következő karakter előrejelzése nélkül csak a perplexitást is számítani. Ez értelemszerűen minimálisan gyorsabb, mindenesetre az OCR-javításhoz, szemétszűréshez, nyelvfelismeréshez és hasonlókhoz csak erre van szükség. Ez a függvény a paddinggel sem szórakozik, mert az általában nem sokat számít, tehát ha pl. OCR-javításra akarjuk használni, akkor a gyanús részsztringet balra legalább n-1 karakter ablakkal célszerű átadni neki.

In [28]:
%%time
print(trie_forward.metrics_on_string(long_string_1))
print(trie_forward.metrics_on_string(long_string_2))

(0.5831381733021077, 3.744253215919672)
(0.5152253349573691, 4.328136190221757)
CPU times: user 650 ms, sys: 0 ns, total: 650 ms
Wall time: 649 ms


In [29]:
%%time
print(trie_forward.string_perplexity(long_string_1))
print(trie_forward.string_perplexity(long_string_2))

3.744253215919672
4.328136190221757
CPU times: user 624 ms, sys: 4.14 ms, total: 628 ms
Wall time: 627 ms


Ha ezt profilozzuk, jól látszik, hogy a futási idő messze legnagyobb részét a `prefix_count()` rekurzív hívásai teszik ki.

Egyébként a jelenlegi már egy teljesítményre optimalizált kód. Amikor első körben megírtam, akkor még vagy ötvenszer ennyi idő alatt futott le, ami használhatatlan volt gyakorlatilag. Sokáig gondolkodtam rajta, hogy hol végez felesleges munkát, és így jutottam el a jelenlegi változathoz.

In [30]:
import cProfile
cProfile.run('trie_forward.metrics_on_string(long_string_1)')

         991967 function calls (650567 primitive calls) in 0.483 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.483    0.483 <string>:1(<module>)
      855    0.001    0.000    0.001    0.000 more.py:832(windowed)
      854    0.062    0.000    0.448    0.001 ngram_model.py:181(estimate_alternatives)
      854    0.010    0.000    0.010    0.000 ngram_model.py:219(<dictcomp>)
      854    0.001    0.000    0.452    0.001 ngram_model.py:249(backoff_alternatives)
        1    0.000    0.000    0.000    0.000 ngram_model.py:288(replace_oov)
        1    0.003    0.003    0.483    0.483 ngram_model.py:303(metrics_on_string)
    97356    0.008    0.000    0.008    0.000 ngram_model.py:347(<lambda>)
439610/98210    0.354    0.000    0.378    0.000 ngram_model.py:98(prefix_count)
        1    0.000    0.000    0.483    0.483 {built-in method builtins.exec}
   443028    0.024    0.000    0.024    0

## Jobbról balra haladó n-gram-modell

A modell megfordított sztringeken lett betanítva, ezért az n-gramokat is megfordítva kell neki odaadni inputként.

In [31]:
trie_backward.estimate_prob('macsk'[::-1])

0.4487024414014663

In [32]:
trie_backward.estimate_alternatives('macsk'[::-1])['m']

0.4487024414014663

In [33]:
trie_backward.predict_next('acsk'[::-1], 5)

['m', 'z', 'M', 'h', 's']

Az előre és a hátra modell perplexitása majdnem azonos mindkét sztringen.

In [34]:
print(trie_forward.metrics_on_string(long_string_1))
print(trie_backward.metrics_on_string(long_string_1[::-1]))

(0.5831381733021077, 3.744253215919672)
(0.5737704918032787, 3.784722241197091)


In [35]:
print(trie_forward.metrics_on_string(long_string_2))
print(trie_backward.metrics_on_string(long_string_2[::-1]))

(0.5152253349573691, 4.328136190221757)
(0.5505481120584653, 4.314408348738489)


Ha elfelejtjük megfordítani az inputot, az azonnal látszik, mert borzalmasan rossz számokat kapunk.

In [36]:
print(trie_backward.metrics_on_string(long_string_2))

(0.0779537149817296, 720.5035030526914)


# Kétirányú n-gram-modell

Két egyirányúból példányosítjuk. Az előre és a hátra modellnek azonos ábécén kell alapulnia, más követelmény nincs, így például konkrétan nem kell azonos rendűnek sem lenniük, lehet pl. előre 5-gram-, hátra 3-gram-modell.

In [37]:
bi_ngram_model = BiMultiNgramModel(trie_forward, trie_backward)

A két modell összekapcsolását a `combine_alternatives()` metódus végzi, ami az egyirányú modellbeli `estimate_alternatives()` megfelelője, és ahhoz hasonlóan működik. Külön megadunk neki egy előre és egy hátra n-gramot (az utóbbit az átadáskor meg kell fordítani), ezeket backoff stb. szempontjából külön-külön feldolgozza a megfelelő egyirányú modell, majd az ezektől kapott két valószínűségi eloszlást a kulcsok (tehát az n-gramok utolsó eleme) szerint páronként összekapcsolja a választott összekapcsoló függvénnyel, majd az eredményt normalizálja valószínűségi eloszlásra. Ha nem adunk meg összekapcsoló függvényt, az alapértelmezés szerint `sum` függvény lesz, tehát az előre és a hátra modell által az egyes karakterekre becsült valószínűségeket átlagoljuk.

In [38]:
bi_ngram_model.combine_alternatives('lakót', 'telep'[::-1])

{'\x00': 0.00013767875595448775,
 'H': 4.242259312272742e-09,
 'V': 8.180856199347752e-06,
 'G': 3.1244289203836156e-09,
 ' ': 0.045867380011428746,
 '9': 1.0408610633787692e-11,
 '/': 9.834252525464875e-06,
 '0': 8.413437666872677e-12,
 '8': 5.3124917666970005e-12,
 '.': 0.0013120381343120753,
 's': 0.0023898830032439175,
 'z': 0.008641838005827941,
 'á': 7.097315442389541e-09,
 'm': 0.00031304039378733244,
 '1': 8.157153039521859e-11,
 'f': 0.00019679037719343346,
 'e': 0.0011014337514695315,
 'b': 0.0047348888910285535,
 'r': 0.0015857417472511215,
 'u': 6.884246211834925e-05,
 '2': 6.960333362839322e-11,
 '7': 5.5179886958907124e-11,
 '\n': 0.00032452848799666443,
 'A': 2.1677185107103502e-10,
 'ű': 3.024826283770442e-09,
 'n': 0.008074118786932432,
 'ü': 9.834232409748387e-06,
 'l': 0.019906080310504202,
 'd': 4.9315066003478194e-05,
 'ö': 0.002104517801375226,
 'é': 0.013266329452048958,
 ',': 0.004553232383189455,
 'a': 0.001986512507355914,
 'y': 1.5088121130771684e-07,
 'k': 0

Mivel a két n-gram összes lehetséges alternatívájára számítjuk a valószínűségeket, ezért az előre n-gram utolsó és a hátra n-gram első karakterére teljesen mindegy, hogy mit írunk be, ugyanazt a valószínűségi eloszlást kapjuk vissza.

In [39]:
bi_ngram_model.combine_alternatives('lakóp', 'zelep'[::-1])

{'\x00': 0.00013767875595448775,
 'H': 4.242259312272742e-09,
 'V': 8.180856199347752e-06,
 'G': 3.1244289203836156e-09,
 ' ': 0.045867380011428746,
 '9': 1.0408610633787692e-11,
 '/': 9.834252525464875e-06,
 '0': 8.413437666872677e-12,
 '8': 5.3124917666970005e-12,
 '.': 0.0013120381343120753,
 's': 0.0023898830032439175,
 'z': 0.008641838005827941,
 'á': 7.097315442389541e-09,
 'm': 0.00031304039378733244,
 '1': 8.157153039521859e-11,
 'f': 0.00019679037719343346,
 'e': 0.0011014337514695315,
 'b': 0.0047348888910285535,
 'r': 0.0015857417472511215,
 'u': 6.884246211834925e-05,
 '2': 6.960333362839322e-11,
 '7': 5.5179886958907124e-11,
 '\n': 0.00032452848799666443,
 'A': 2.1677185107103502e-10,
 'ű': 3.024826283770442e-09,
 'n': 0.008074118786932432,
 'ü': 9.834232409748387e-06,
 'l': 0.019906080310504202,
 'd': 4.9315066003478194e-05,
 'ö': 0.002104517801375226,
 'é': 0.013266329452048958,
 ',': 0.004553232383189455,
 'a': 0.001986512507355914,
 'y': 1.5088121130771684e-07,
 'k': 0

Tetszőleges függvényt megadhatunk, ami kételemű listából értelmezhető numerikus eredményt ad, pl. össze is szorozhatjuk a két külön becsült valószínűséget, vagy számolhatjuk a harmonikus közepüket.

In [40]:
multiply = lambda x: x[0] * x[1]
bi_ngram_model.combine_alternatives('lakóp', 'zelep'[::-1], combine_function=multiply)

{'\x00': 1.337648372080904e-13,
 'H': 4.763290958722882e-18,
 'V': 1.7772330328685344e-13,
 'G': 2.6308211943262398e-18,
 ' ': 0.00013460366280637813,
 '9': 1.139846206354772e-21,
 '/': 2.382409237517969e-14,
 '0': 7.299507725424648e-22,
 '8': 2.5688652187552125e-22,
 '.': 2.2612748870452182e-07,
 's': 1.753206000848735e-08,
 'z': 2.4945432438140932e-05,
 'á': 1.930312705443978e-17,
 'm': 1.0541281428331091e-07,
 '1': 1.0593410586522519e-20,
 'f': 8.851758018383823e-10,
 'e': 1.8099766248432653e-10,
 'b': 0.00015362387495778778,
 'r': 8.160992073546652e-07,
 'u': 9.007574626359783e-12,
 '2': 4.480774742222206e-21,
 '7': 5.190371118222622e-21,
 '\n': 6.309805753100165e-13,
 'A': 4.948013424223668e-19,
 'ű': 1.7783608183758933e-18,
 'n': 8.344761283819867e-08,
 'ü': 1.5462369920181e-14,
 'l': 1.653280977565993e-05,
 'd': 2.9947730512778723e-10,
 'ö': 2.3121298691665802e-12,
 'é': 6.222371220472896e-11,
 ',': 6.164977953717139e-17,
 'a': 4.260949259843597e-10,
 'y': 5.576327872599458e-16,

In [41]:
harmonic_mean = lambda x: 2 / (1 / x[0] + 1 / x[1])
bi_ngram_model.combine_alternatives('lakóp', 'zelep'[::-1], combine_function=harmonic_mean)

{'\x00': 4.791232883040705e-10,
 'H': 5.537097200823365e-10,
 'V': 1.0713166790851864e-08,
 'G': 4.1523402329463514e-10,
 ' ': 0.0014471888004790763,
 '9': 5.400397703609621e-11,
 '/': 1.1946680711752337e-09,
 '0': 4.278507912901896e-11,
 '8': 2.3845986685818305e-11,
 '.': 8.499221549475826e-05,
 's': 3.6176667341664355e-06,
 'z': 0.0014234989935525758,
 'á': 1.341237877518072e-09,
 'm': 0.00016606009247757954,
 '1': 6.404267201245744e-11,
 'f': 2.21818581337544e-06,
 'e': 8.103764918485307e-08,
 'b': 0.016000043802501005,
 'r': 0.00025379481465717755,
 'u': 6.452436260046918e-08,
 '2': 3.174646107970814e-11,
 '7': 4.638631294294664e-11,
 '\n': 9.588163074259369e-10,
 'A': 1.125641850685129e-09,
 'ű': 2.8992902269799735e-10,
 'n': 5.09672300637434e-06,
 'ü': 7.753679402490972e-10,
 'l': 0.0004095747411375496,
 'd': 2.9947222286457543e-06,
 'ö': 5.417911029963247e-10,
 'é': 2.3130093448300056e-09,
 ',': 6.67703973024538e-15,
 'a': 1.0577604078805957e-07,
 'y': 1.822574911015894e-09,
 'k

A kényelmi metódusok lényegében megegyeznek az egyirányú modellben rendelkezésre állókkal, de a paramétereik eltérnek annak következtében, hogy kétirányú a modell.

A `predict_next()` metódusnak a keresett középső elemtől balra és jobbra eső (n-1)-gramot kell átadni, és az utóbbit meg kell fordítani.

In [42]:
print(bi_ngram_model.predict_next('lakó', 'elep'[::-1]))
print(bi_ngram_model.predict_next('egol', 'ást '[::-1], m=4))

['t']
['d', 't', 'l', 'z']


Az `estimate_prob()` metódus ugyanúgy képes kezelni olyan helyzeteket, amikor rövidebb vagy hosszabb bal vagy jobb n-gramot adunk át neki a két egyirányú modell rendjénél, mint ahogy a metódus egyirányú megfelelője is tudja ezeket kezelni, viszont ahhoz, hogy tudja, hogy melyik karakter valószínűségét kell megbecsülnie, ilyen helyzetekben meg kell adni a kérdéses elem indexét második vagy nevesített argumentumként.

Ha ezt nem tesszük meg, alapértelmezés szerint az n-edik elem valószínűségét becsüli meg, ahol n az előre modell rendje.

In [43]:
print(len('egoldást '))
print(bi_ngram_model.estimate_prob('egoldást '))
print(bi_ngram_model.estimate_prob('egoldást ', 4)) # d
print(bi_ngram_model.estimate_prob('egoldást ', 3)) # l
print(bi_ngram_model.estimate_prob('egoldást ', 2)) # o

print(bi_ngram_model.estimate_prob('macskát', 3)) # s
print(bi_ngram_model.estimate_prob('macskát')) # k

9
0.5419201827236841
0.5419201827236841
0.45670667893237255
0.47380608207172187
0.14087125462165861
0.4133456382486001


A `metrics_on_string()` metódus hasonlóan egy ablakot léptet végig a sztringet, és lépésenként az előre és hátra n-gram metszéspontjában levő karaktert értékeli ki.

Ha van padding, akkor nemcsak az elején értékeli ki az első karaktert a padding utáni bigram, a másodikat a trigram stb. alapján, hanem ugyanezt csinálja fordítva a sztring végén is. Ha nincs padding, akkor simán teljes ablakokat léptet.

In [44]:
%%time
print(trie_forward.metrics_on_string(long_string_1))
print(trie_backward.metrics_on_string(long_string_1[::-1]))

(0.5831381733021077, 3.744253215919672)
(0.5737704918032787, 3.784722241197091)
CPU times: user 649 ms, sys: 3.88 ms, total: 653 ms
Wall time: 651 ms


In [45]:
%%time
print(bi_ngram_model.metrics_on_string(long_string_2))

(0.7356181150550796, 2.982173827104181)
CPU times: user 710 ms, sys: 7.91 ms, total: 718 ms
Wall time: 716 ms


Perplexitások különböző összekapcsoló függvényekre. Úgy tűnik, ezek közül következetesen a **szorzás** adja a legjobb eredményeket mind pontosságra, mind perplexitásra nézve, illetve a pontossága nyilván megegyezik a mértani középével. A tanulság az, hogy tényleg érdemes különböző összekapcsoló függvényekkel kísérletezni.

In [46]:
geometric_mean = lambda x: (x[0] * x[1]) ** 2
max_squared = lambda x: max(x) ** 2

In [47]:
print(bi_ngram_model.metrics_on_string(long_string_1)) # sum
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=max))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=max_squared))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=geometric_mean))
print(bi_ngram_model.metrics_on_string(long_string_1, combine_function=harmonic_mean))

(0.8058823529411765, 2.630060632382658)
(0.7705882352941177, 2.928863960323583)
(0.7705882352941177, 2.1839104337061452)
(0.8623529411764705, 1.5094330610760718)
(0.8623529411764705, 1.7145876208025843)
(0.8247058823529412, 1.7446173300756533)


In [48]:
print(bi_ngram_model.metrics_on_string(long_string_2)) # sum
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=max))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=max_squared))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=geometric_mean))
print(bi_ngram_model.metrics_on_string(long_string_2, combine_function=harmonic_mean))

(0.7356181150550796, 2.982173827104181)
(0.6988984088127295, 3.322779019968607)
(0.6988984088127295, 2.609475595243155)
(0.7968176254589964, 1.7982801627641019)
(0.7968176254589964, 2.273808259070984)
(0.7723378212974297, 2.052976764170971)


Azt is kiírathatjuk, hogy az egész sztringben melyik karaktereket jelzi rendre előre legvalószínűbbként a modell:

In [49]:
print(long_string_1)

A „Tarthatatlan a parkolási helyzet a lakótelepeken” című cikkünkben részletesen bemutattuk, hogy a Budapest sűrűn lakott városrészeiben élő autósok számára egyre többször okoz elemi problémát a parkolóhely-keresés. A folytatásban Ujj Péter Tamás a lehetséges megoldásokat mutatja be. Összefoglalja, mi várható a jövőben, illetve jár-e egyáltalán a parkolóhely az egyre szaporodó autók számára a közterületeken, vagy lassan fel kell hagynunk az eddigi életmódunkkal.
Néhány hete részletes cikkben foglalkoztunk az Óbudán kialakult parkolási káosszal, miután kiderült, hogy már a helyi közterület-felügyelet sem minden szabálytalan autót büntet meg, annyira elfogytak a helyek. A cikk megjelenését követően záporoztak az olvasói levelek, és ahogy várható volt, rengetegen szeretnék, ha az érintett hivatalok végre megoldást találnának a súlyosbodó problémára.


In [50]:
bi_ngram_model.metrics_on_string(long_string_1, print_string=True, padded=True)

A  talthatatlan a puslolósa helyeet a lakoteletetet  című cikkünk en részzeteten bemutatjak, hogy a Budapest sűrűs lakost várásrészetben eső autósok számára egyre többször ohaz lllm  problémák a purkol hály keresig. A folytatásban UUh méter Tamás m levetségit megoldások t mutat a ae. összefoglalka  hi válható a jövőben  ellette aátte egyáltat l a purkol mely a  egy e szaborodá aatók számára a ks kereletebet  vagy laosan vel kell hagytink a  eddig  életmudunk al. némány lele részzetet ciekeen foglalkoztank a  Ózudan kialakult puslolósa khrszzat, m után kikerült, hogy más a helye ks kerelet feligyelet nem minden szabolytttan a tót bentet megt ennyite e fogaai  a helyet. A cikk megjelenéret követben zarorodaat a  olvashi lehenett és  hogy válható volt   endedeget szerették, ho a  erent tt hivatalos végte megoldást taráltinak a súlyoskídi problémál k


(0.8018648018648019, 2.7088778690523965)

In [51]:
bi_ngram_model.metrics_on_string(long_string_1, combine_function=multiply, print_string=True, padded=True)

A ktarthatatlan a partolása helyzet a lakiteletetet, című cikkünkben részletesen bemutatjak, hogy a budapest serűs lakost városrészetben eső autósok számára egyre többször okaz ellmb problémák a parkolókály keretég. A folytatásban ajh Péter Tamás m levetséget megoldásokat mutat a be. Összefoglalta, aa válható a jövőben, ellette aátte egyáltatál a parkolókely az egyse szaborodt aztók számára a köztereletetet, vagy lassan vel kell hagytink az eddigi lletlódunkkal. Némány keli részletet cikkben foglalkottank az ezudan kialakula partolása kroszzat, miután kikerült, hogy más a helye közterelet felegyeket nem minden szabolytatan aztót tentet megt annyita elfogltik a helyet. A ciák megjelenéset követsen zasorodnat az olvasni lesenett és ahogy válható volta hengedegek szerették, ha az erintott hivatalos végre megoldást tanáltinak a súlyossodi problémál K


(0.8578088578088578, 1.5973799384302159)

## OCR-hibák, nem magyar szöveg

In [52]:
string1_correct = "A Matáv Adományvonal lehetővé teszi, hogy ön egyszerűen és gyorsan juttasson célba adományt, míg a nonprofit szervezetek pályázati úton, rövid idő alatt juthatnak nagyobb összegekhez egészségmegőrző, szociális es gyógyító programjaikhoz."
string1_ocr = "A MatPv AdomPny vonal /hl^'etVvö tesiri, hgyy ön egyszerűen és gyorsan UittUssson cáia adományt, míg a non-profit szervezetek pályázati úton, dVvdJ kföalatt uutaatnak nagobür összegekhez ^fS^^e^í^zö, tooPPHs és gyógyító'programjaihhzz."
print(string1_correct)
print(string1_ocr)

A Matáv Adományvonal lehetővé teszi, hogy ön egyszerűen és gyorsan juttasson célba adományt, míg a nonprofit szervezetek pályázati úton, rövid idő alatt juthatnak nagyobb összegekhez egészségmegőrző, szociális es gyógyító programjaikhoz.
A MatPv AdomPny vonal /hl^'etVvö tesiri, hgyy ön egyszerűen és gyorsan UittUssson cáia adományt, míg a non-profit szervezetek pályázati úton, dVvdJ kföalatt uutaatnak nagobür összegekhez ^fS^^e^í^zö, tooPPHs és gyógyító'programjaihhzz.


In [53]:
print(bi_ngram_model.metrics_on_string(string1_correct))
print(bi_ngram_model.metrics_on_string(string1_ocr))

(0.8209606986899564, 2.564246492735871)
(0.46255506607929514, 25.62730166071084)


Összeadással is szépen különválik a jó és az OCR-hibás szöveg perplexitása, de szorzással még jobban:

In [54]:
print(bi_ngram_model.metrics_on_string(string1_correct, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(string1_ocr, combine_function=multiply))

(0.8384279475982532, 1.5190116169969372)
(0.5022026431718062, 319.6578162054826)


In [55]:
string2_good_ocr = "Környezetvédelem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du- - A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól.	a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése)	".replace('\t', ' ')
string2_bad_ocr = "K^t^r^yt^zz^t^i^tédielem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du-	- A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól.	a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése)	".replace('\t', ' ')
print(string2_good_ocr)
print(string2_bad_ocr)

Környezetvédelem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du- - A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól. a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése) 
K^t^r^yt^zz^t^i^tédielem, közlekedés - Budapest fuldoklik a szennyezéstől és a közlekedési du- - A főváros mindent megtesz azért, hogy vonzóbbá tegye góktól. a tömegközlekedést (a 4-es és 5-ös metró megépítésének kezdeményezése) 


In [56]:
print(bi_ngram_model.metrics_on_string(string2_good_ocr))
print(bi_ngram_model.metrics_on_string(string2_bad_ocr))

(0.7605633802816901, 3.1477087801617234)
(0.6877828054298643, 5.251298562043095)


In [57]:
print(bi_ngram_model.metrics_on_string(string2_good_ocr, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(string2_bad_ocr, combine_function=multiply))

(0.8169014084507042, 2.148832287267806)
(0.746606334841629, 6.035245541728297)


Az alábbiak azok a rövid OCR-hibás sztringek, amikre panaszkodtam a 15-ös issue-ban, hogy a nyelvfelismerők kiváló magyar szövegként azonosítják őket, illetve még rosszabbnak látják a hibátlan magyar szöveget, mint a rontottat. A nyelvmodell szépen jelzi, hogy a hibás szöveg nem magyar, míg a helyes szöveg az.

In [58]:
string3_correct = "értékesítő iroda nyitvatartása: hétfő-péntek 8-19h"
string3_ocr = "értíkesítto Iroda m'itfatartdsa: Mjü-ln)Hl<rk 8-19b"

print(bi_ngram_model.metrics_on_string(string3_correct, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(string3_ocr, combine_function=multiply))

(0.8095238095238095, 2.2938187516414454)
(0.18604651162790697, 3158.0590342045866)


In [65]:
string4 = "az Átadva az  to^a KeaeáéNt AaaéWkéFedAa cat (atwNtaaA. aww^y"
string5 = "A a L ét • a  ffc Mkm bML A hWiWWHK1**(re a nMtjjww áHő"

print(bi_ngram_model.metrics_on_string(string4, combine_function=multiply))
print(bi_ngram_model.metrics_on_string(string5, combine_function=multiply))

(0.16981132075471697, 307082.82207396714)
(0.14893617021276595, 480687.77713892626)


Nem magyar nyelvű szöveg felismerésére is jól használhatók a modellek. Míg a magyar szöveg karakterenkénti perplexitása összekapcsoló függvénytől függően 1-4 körül mozog, az angolé 10 felett van. Látványos, hogy a nem emberi nyelven írt OCR-hibás szemét drasztikusan rosszabb perplexitást kap, mint az emberi nyelven írt szöveg.

In [60]:
string_en = "Day Three of PAX East 2022 is done and dusted for another year, which means it's time for another daily round-up video. We had another jam-packed schedule today, checking out Dome Keeper (formerly known as Ludum Dare darling Dome Romantik) over on the Raw Fury booth, before heading over to the PAX Rising Showcase to check out a specially curated selection of upcoming indie games."
print(string_en)
print(bi_ngram_model.metrics_on_string(string_en))
print(bi_ngram_model.metrics_on_string(string_en, combine_function=multiply))

Day Three of PAX East 2022 is done and dusted for another year, which means it's time for another daily round-up video. We had another jam-packed schedule today, checking out Dome Keeper (formerly known as Ludum Dare darling Dome Romantik) over on the Raw Fury booth, before heading over to the PAX Rising Showcase to check out a specially curated selection of upcoming indie games.
(0.37967914438502676, 10.597837912956956)
(0.44919786096256686, 16.246758630922756)


Végül a szemétszűrő. Soronként előbb megnézzük, hogy melyik milyen perplexitást kap.

In [61]:
long_ocr_string = '''HIRDETÉS
Fotó: MTI
UTOLSÓ HELYEINK
FANTASZTIKUS KÓ/no tours
Fialj Janos Petrányi Judit Rónai Egon Nagy ibolya Bornemissza Tamas
í •f&lMíh•ErKft	iíföfaíj<4^410
TUNÉZIA -október
X nap/7 éj IO nap/9 éj hotel*** fp.-vnl 59 900 I t/ífttöi 74 900 I vrőtól hotel**** fp.-vwl 74 900 It/fölől X4 900 JVfőtrtl hotel.....n».-vnl X9 900 It/fótól 99 900 I VfMól
DJERBA - október
üdülés, hotel*•• fp.-val:	79 900 Fl/főtől
korutuzas ♦ üdülés Djerba szigetén,
hotel***:	114 900 Ft/főt ől
HURGHADA -október
X nap hottf**** fp.-val	69 900 H/fótól
botéi***** fp.-val	96 900 Fl/főtől
Nílusi hajóul közvetlen chalerjarultal. 5* m luxuslupóval: 129 900 Ftíőtől
TH-: 2M»-O<.*2. 2M-OH33 TH.: .M2-O.MO. .152-WM* MTI. l«Mr«l 44M. nd.
ajánlaton Portugália .bombaáron* 'épülő, 7 nap teljes p.. X. 14-20.. 21-27 189 900 Ft/« DáFSpanyolorazág-Marokkó. rep. 13 nap
3/4* száll, fél. ill, teljes p. X. 28-XI 9. 349 900 Ft/« Prága-Kariovy Vary, busz 5 nap fp„ X. 19-23.39 900-47 900 Ft/« München, Sörfesztivál, busz, 3 nap reggeli, IX. 27-29.. X. 4-6 29 900 R/«
Cseh várak, városok ét kastélyok, busz, 7 nap reggelt. X. 22-28.. 54 900 Ft/R Rótna-Nápoly-Pompejl, busz, 6 nap reggeli, X, 22-27 66 900 R/W Advent Steyrben, busz, 2 nap fp„ XI. 30.. XII. 1 . XII. 14 . 25 900 R/K Szilveszter az OPATIJAI RIVIERÄN busz, 4 nap fp„ XII. 30 53 900 Ft/« ♦ kdt. szilv. vacsora ' jelentkezés' szekszárdi irodánknál (74)* 5ÍV-955; és több száz vidéki és budapesti partnerünknél toltour@toln8.net, www.toltour.lolna.net
Eng. szám R01963 1999
Prwim Póton Laak
– Fittségi kompiét tesztelés (aUckepevseg. «leti mozjékoaysag)
– Csoportot es egyéni tiiucsadzs (alzklonnilzsrol, testi panaszok- ml eWmodroi, helyes táplálkozásról)
TAéton pArari, hi^wlóáá. ftáadun bntfto. ma. euaa. ftanz^UKKJ.
HASTAMCTANULÁ8 |«t «rakat Joli Jtott	tetor tartja)
Hbrlmcv krtagm ahaayoMi trabaUaa
PEJKÓ Lovaspanzió. Csongrád Mobd: 106-201921-8004«Tel !ai. (06-1) 349-3650
* A szeptemberi időponthoz további 10% engedmény'
A Kereskedelmi és Hitelbank Rt letétkezelésében lévő alapok nettó eszközérték Ft (egy jegy re jutó nettó eszközérték) adatai:
2002. október 4-én:
Aranyhal 359 906 570 (2,584)
2002. október 2-án:
Első Hazai Lakásalap 51 789 494 (9 691)
Mély fájdalommal tudatjuk mindazokkal, akik ismerték és szerették, hogy KÜSTEL RICHÁRD oki. elektromérnök, közgazdász életének 78. évében, hősiesen viselt súlyos betegségben 2002. szeptember 19-én elhunyt.
Végső búcsúztatása a Farkasréti temetőben lesz október 10-én 10.30 órakor. A gyászoló család.
A szeptember 28-al keresztrejtvény nyertese:
Zics Lajos, 1135 Budapest
Nyereményét, a Nagy Képes Világőrténet, és a Magyarország családjai CD-Romot, postázzuk!
Gratulálunk a nyereményhez!'''

In [62]:
for line in long_ocr_string.split('\n'):
    if len(line) < 9:
        continue
    print(line)
    print('\t', bi_ngram_model.metrics_on_string(line, combine_function=multiply))

Fotó: MTI
	 (0.0, 2.6380338031396504)
UTOLSÓ HELYEINK
	 (0.42857142857142855, 33.301556438236425)
FANTASZTIKUS KÓ/no tours
	 (0.3125, 103.46043610157908)
Fialj Janos Petrányi Judit Rónai Egon Nagy ibolya Bornemissza Tamas
	 (0.6271186440677966, 6.3401560317263606)
í •f&lMíh•ErKft	iíföfaíj<4^410
	 (0.045454545454545456, 47118715.74202395)
TUNÉZIA -október
	 (0.5, 12.210995846406016)
X nap/7 éj IO nap/9 éj hotel*** fp.-vnl 59 900 I t/ífttöi 74 900 I vrőtól hotel**** fp.-vwl 74 900 It/fölől X4 900 JVfőtrtl hotel.....n».-vnl X9 900 It/fótól 99 900 I VfMól
	 (0.2926829268292683, 1401.7067711706945)
DJERBA - október
	 (0.625, 14.955936618620552)
üdülés, hotel*•• fp.-val:	79 900 Fl/főtől
	 (0.3939393939393939, 1304.1916205091559)
korutuzas ♦ üdülés Djerba szigetén,
	 (0.4444444444444444, 64.1692913931725)
hotel***:	114 900 Ft/főt ől
	 (0.3684210526315789, 1174.0245871020159)
HURGHADA -október
	 (0.3333333333333333, 31.23847172618334)
X nap hottf**** fp.-val	69 900 H/fótól
	 (0.2, 2000.3507414

Ezek alapján legyen mondjuk 5 a küszöbérték.

In [63]:
for line in long_ocr_string.split('\n'):
    if len(line) < 9:
        continue
    perpl = bi_ngram_model.metrics_on_string(line, combine_function=multiply)[1]
    if perpl < 5:
        print(line)
        print('\t', perpl)

Fotó: MTI
	 2.6380338031396504
* A szeptemberi időponthoz további 10% engedmény'
	 1.5648542356726869
A Kereskedelmi és Hitelbank Rt letétkezelésében lévő alapok nettó eszközérték Ft (egy jegy re jutó nettó eszközérték) adatai:
	 3.0391442471613606
2002. október 4-én:
	 1.439625993835486
2002. október 2-án:
	 1.1880397121979542
Mély fájdalommal tudatjuk mindazokkal, akik ismerték és szerették, hogy KÜSTEL RICHÁRD oki. elektromérnök, közgazdász életének 78. évében, hősiesen viselt súlyos betegségben 2002. szeptember 19-én elhunyt.
	 2.479004951096471
Végső búcsúztatása a Farkasréti temetőben lesz október 10-én 10.30 órakor. A gyászoló család.
	 1.6209947169020056
A szeptember 28-al keresztrejtvény nyertese:
	 2.0855193054917556
Zics Lajos, 1135 Budapest
	 2.4226017658186008
Nyereményét, a Nagy Képes Világőrténet, és a Magyarország családjai CD-Romot, postázzuk!
	 3.661845271876318
Gratulálunk a nyereményhez!
	 1.3019049228616995


In [64]:
for line in long_ocr_string.split('\n'):
    if len(line) < 9:
        continue
    perpl = bi_ngram_model.metrics_on_string(line, combine_function=multiply)[1]
    if perpl > 5:
        print(line)
        print('\t', perpl)

UTOLSÓ HELYEINK
	 33.301556438236425
FANTASZTIKUS KÓ/no tours
	 103.46043610157908
Fialj Janos Petrányi Judit Rónai Egon Nagy ibolya Bornemissza Tamas
	 6.3401560317263606
í •f&lMíh•ErKft	iíföfaíj<4^410
	 47118715.74202395
TUNÉZIA -október
	 12.210995846406016
X nap/7 éj IO nap/9 éj hotel*** fp.-vnl 59 900 I t/ífttöi 74 900 I vrőtól hotel**** fp.-vwl 74 900 It/fölől X4 900 JVfőtrtl hotel.....n».-vnl X9 900 It/fótól 99 900 I VfMól
	 1401.7067711706945
DJERBA - október
	 14.955936618620552
üdülés, hotel*•• fp.-val:	79 900 Fl/főtől
	 1304.1916205091559
korutuzas ♦ üdülés Djerba szigetén,
	 64.1692913931725
hotel***:	114 900 Ft/főt ől
	 1174.0245871020159
HURGHADA -október
	 31.23847172618334
X nap hottf**** fp.-val	69 900 H/fótól
	 2000.3507414559447
botéi***** fp.-val	96 900 Fl/főtől
	 2039.3918782027094
Nílusi hajóul közvetlen chalerjarultal. 5* m luxuslupóval: 129 900 Ftíőtől
	 55.57413745768618
TH-: 2M»-O<.*2. 2M-OH33 TH.: .M2-O.MO. .152-WM* MTI. l«Mr«l 44M. nd.
	 119269.42627733572
a

Ez így szerintem kielégítő eredmény. Úgy tűnik, valami nem jó a padding opcióban, nagyon elrontja a padding nélkül hibátlannak ítélt sorok mért perplexitását is. Bug lehet benne valahol, majd keressem meg alkalomadtán.